<a href="https://colab.research.google.com/github/razor3598/FPL-Visualization/blob/main/fpl_Teams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Choose type of Plot and Enter League code { display-mode: "form" }

plot_type = "Cumulative Points" #@param ["Cumulative Points", "Individual Gameweek Points"]
league_code =  1254781#@param {type:"integer"}
import requests
import numpy as np
import sys
import plotly.express as px

'''
Finding out the number of gameweeks played in real time.
'''
resp = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
try:
  data_gen = resp.json()
except:
  print("\nError occured in fetching data.")
  !kill -9 -1

gameweeks = 1
for event in data_gen['events']:
  if event['finished'] == True:
    gameweeks += 1

  else:
    break
###################################################################################################################################################################

'''
Fetching data from the league.
'''

resp = requests.get("https://fantasy.premierleague.com/api/leagues-classic/{league_code}/standings/".format(league_code = league_code))

try:
  data = resp.json()
except:
  print("\nError occured in fetching data.")
  sys.exit()

manager_ids = []
manager_names = []

league_name = data['league']['name']

if len(data['standings']['results']) < 10:
  num_players = len(data['standings']['results'])
else:
  num_players = 10

# Getting manager id and names
for i in range(num_players):
  manager_ids.append(data['standings']['results'][i]['entry'])
  manager_names.append(data['standings']['results'][i]['entry_name'])

# League Name


def choose_plot():
  count = 0

  manager_info = {}
  for manager_id in manager_ids:
    resp = requests.get("https://fantasy.premierleague.com/api/entry/{manager_id}/history/".format(manager_id = manager_id))

    try:
      data = resp.json()
    except:
      print("\nError occured in fetching data.")
      !kill -9 -1

    manager_info[manager_names[count]] = {}

    if plot_type == "Cumulative Points":
      for entry in data['current']:
        arr = manager_info[manager_names[count]]
        arr[entry['event']] = entry['total_points']
      count += 1

    elif plot_type == "Individual Gameweek Points":
      for entry in data['current']:
        arr = manager_info[manager_names[count]]
        arr[entry['event']] = entry['points']
      count += 1

  return manager_info

###################################################################################################################################################################
'''
Plotting the data
'''

manager_info = choose_plot()

fig = px.line(manager_info, title= plot_type + ' vs Gameweeks - League:' + league_name, markers = True)
fig.show()